In [1]:
import os
import time
import re
import pandas as pd
import numpy as np

In [2]:
start = time.time()
fp = '/Users/johnmichael/Documents/DATA514/project/data/'
with open(os.path.join(fp, 'amazon-meta.txt'), 'r') as f:
    meta = f.readlines()
with open(os.path.join(fp, 'amazon0302.txt'), 'r') as f:
    graph_0302 = f.readlines()
with open(os.path.join(fp, 'amazon0312.txt'), 'r') as f:
    graph_0312 = f.readlines()
with open(os.path.join(fp, 'amazon0505.txt'), 'r') as f:
    graph_0505 = f.readlines()
with open(os.path.join(fp, 'amazon0601.txt'), 'r') as f:
    graph_0601 = f.readlines()

In [3]:
# Create groups table
groups = pd.DataFrame({'group_id': ['gid_283155', 'gid_5174', 'gid_130', 'gid_404272'],
                       'group_name': ['Book', 'Music', 'DVD', 'Video']})

In [4]:
n_products = 548552 # from documentation
n_reviews = 7781990 # from documentation

# products table variables
product_id = [None] * n_products
group_name = [None] * n_products
asin = [None] * n_products
title = [None] * n_products
salesrank = [None] * n_products
categories = [None] * n_products
discontinued = [None] * n_products

# reviews table variables
review_product_id = [None] * n_reviews
review_date = [None] * n_reviews
customer = [None] * n_reviews
rating = [None] * n_reviews
votes = [None] * n_reviews
helpful = [None] * n_reviews

curr_product_id = None
curr_group_name = None
curr_asin = None
curr_title = None
curr_salesrank = None
curr_categories = None
curr_review_product_id = None
curr_review_date = None
curr_customer = None
curr_rating = None
curr_votes = None
curr_helpful = None
product_counter = 0
review_counter = 0
for i, line in enumerate(meta):
    if i % 1e6 == 0:
        print('Line: ' + str(i))
    if i == len(meta) - 1 or re.match('Id:', line):
        # append lists if line reaches a new product or the end of file
        if curr_product_id is not None:
            product_id[product_counter] = curr_product_id
            group_name[product_counter] = curr_group_name
            asin[product_counter] = curr_asin
            title[product_counter] = curr_title
            salesrank[product_counter] = curr_salesrank
            categories[product_counter] = curr_categories
            product_counter += 1
        curr_product_id = 'pid_' + re.sub('Id:', '', line).strip()

    if re.match('ASIN:', line):
        curr_asin = 'asin_' + re.sub('ASIN:', '', line).strip()
        
    if re.search('title:', line):
        curr_title = re.sub('title:', '', line).strip()
    
    if re.search('group:', line):
        curr_group_name = re.sub('group:', '', line).strip()
        
    if re.search('salesrank:', line):
        curr_salesrank = re.sub('salesrank:', '', line).strip()
        
    if re.search('categories:', line):
        curr_categories = re.sub('categories:', '', line).strip()
        
    if re.search('\d{4}-\d{1,2}-\d{1,2}', line):
        curr_review_product_id = curr_product_id
        curr_review_date = re.search('\d{4}-\d{1,2}-\d{1,2}', line).group(0)
        curr_customer = re.search('cutomer:\s+([A-Z]|\d)+\s', line).group(0)
        curr_customer = re.sub('cutomer:', '', curr_customer).strip()
        curr_rating = re.search('rating:\s+\d+\s', line).group(0)
        curr_rating = re.sub('rating:', '', curr_rating).strip()
        curr_votes = re.search('votes:\s+\d+\s', line).group(0)
        curr_votes = re.sub('votes:', '', curr_votes).strip()
        curr_helpful = re.search('helpful:\s+\d+', line).group(0)
        curr_helpful = re.sub('helpful:', '', curr_helpful).strip()
        review_product_id[review_counter] = curr_review_product_id
        review_date[review_counter] = curr_review_date
        customer[review_counter] = curr_customer
        rating[review_counter] = curr_rating
        votes[review_counter] = curr_votes
        helpful[review_counter] = curr_helpful
        review_counter += 1
    
    if re.search('discontinued product', line):
        discontinued[product_counter] = 1


Line: 0
Line: 1000000
Line: 2000000
Line: 3000000
Line: 4000000
Line: 5000000
Line: 6000000
Line: 7000000
Line: 8000000
Line: 9000000
Line: 10000000
Line: 11000000
Line: 12000000
Line: 13000000
Line: 14000000
Line: 15000000


In [5]:
products = pd.DataFrame({'product_id': product_id, 'group_name': group_name,
                         'asin': asin, 'title': title, 'salesrank': salesrank,
                         'categories': categories, 'discontinued': discontinued})

reviews = pd.DataFrame({'review_product_id': review_product_id,
                        'review_date': review_date,
                        'customer': customer, 'rating': rating,
                        'votes': votes, 'helpful': helpful})

for i, value in enumerate(products['group_name']):
    if value in ['Baby Product', 'Video Games']:
        products.loc[i, 'group_name'] = 'Book'
    if value == 'CE': 
        if products.loc[i, 'title'] == 'SPELLING CORRECTOR':
            products.loc[i, 'group_name'] = 'Book'
        else:
            products.loc[i, 'group_name'] = 'Video'
    if value == 'Software': 
        if products.loc[i, 'title'] == 'Just Enough Vocals The Learning Co':
            products.loc[i, 'group_name'] = 'DVD'
        else:
            products.loc[i, 'group_name'] = 'Book'
    if value == 'Sports':
        products.loc[i, 'group_name'] = 'Video'
    if value == 'Toy':
        if re.search('Songs|Karaoke', products.loc[i, 'title']):
            products.loc[i, 'group_name'] = 'Music'
        else:
            products.loc[i, 'group_name'] = 'Book'
    
products = products.merge(groups, on='group_name', how='left')
products.drop(columns=['group_name'], inplace=True)
reviews.dropna(inplace=True)
reviews.reset_index(inplace=True)
reviews['review_date'] = pd.to_datetime(reviews['review_date']).dt.date

In [6]:
def extract_graph(graph_lines, collection_date):
    from_id = [None] * len(graph_lines)
    to_id = [None] * len(graph_lines)
    for i, line in enumerate(graph_lines):
        if re.match('\d', line):
            line_split = line.split('\t')
            from_id[i] = 'pid_' + line_split[0].strip()
            to_id[i] = 'pid_' + line_split[1].strip()
    graph = pd.DataFrame({'from_id': from_id,
                          'to_id': to_id,
                          'collection_date': collection_date})
    graph.dropna(inplace=True)
    graph.reset_index(inplace=True)
    return graph

graphs = extract_graph(graph_0302, '2003-03-02')
graphs = pd.concat([graphs, extract_graph(graph_0312, '2003-03-12')],
                   ignore_index=True)
graphs = pd.concat([graphs, extract_graph(graph_0505, '2003-05-05')],
                   ignore_index=True)
graphs = pd.concat([graphs, extract_graph(graph_0601, '2003-06-01')],
                   ignore_index=True)

In [7]:
products.to_csv(os.path.join(fp, 'clean/products.csv'), index=False)
groups.to_csv(os.path.join(fp, 'clean/groups.csv'), index=False)
reviews.to_csv(os.path.join(fp, 'clean/reviews.csv'), index=False)
graphs.to_csv(os.path.join(fp, 'clean/graphs.csv'), index=False)

In [8]:
print('Time elapsed:', end = ' ')
print(time.time() - start)

Time elapsed: 197.7379550933838
